# AI Business Brochure Generator

- This project use Multi-shot prompting where you provide multiple examples of input-output pairs to help an AI model better understand the pattern or task you want it to perform. Instead of just giving one example (single-shot) or no examples (zero-shot).
- This function allows user to select different language and tones

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr 

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
website_provided = Website("https://www.brooklynkids.org/")

In [6]:
website_provided.links

['#content',
 '/calendar',
 'https://www.brooklynkids.org',
 'https://www.brooklynkids.org/visit/',
 'https://www.brooklynkids.org/visit/',
 'https://www.brooklynkids.org/faq/',
 '/exhibit-type/permanent/',
 '/programs/',
 'https://www.brooklynkids.org/colorlab/',
 'https://www.brooklynkids.org/natures-engineers/',
 '/events/',
 'https://www.brooklynkids.org/movies/',
 'https://www.brooklynkids.org/movies/',
 'https://www.brooklynkids.org/saturday-morning-cartoons/',
 'https://www.brooklynkids.org/birthdays/',
 'https://www.brooklynkids.org/birthdays/',
 'https://www.brooklynkids.org/space-rentals/',
 'https://www.brooklynkids.org/education/',
 'https://www.brooklynkids.org/education/',
 'https://www.brooklynkids.org/school-visits/',
 'https://www.brooklynkids.org/camp-visits/',
 'https://www.brooklynkids.org/summer-camp/',
 'https://www.brooklynkids.org/museum-on-the-go/',
 'https://www.brooklynkids.org/after-school-program/',
 'https://www.brooklynkids.org/teens/',
 'https://www.broo

# Link selection section
### As you can see there are many link in a given website, let's create a block of fucntion to ask GPT which link is relevant


In [7]:
# create a prompt to allow GPT to decide which link from the website is relevant to create a company brochure
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages or Pricing of product/service.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

# create brochure function

In [8]:
# create a function to get details from the links selected by get_links
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result
    
# and then those details will be compose into user prompt to create brochure
def get_brochure_user_prompt(company_name, url, language="English"):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; \
    use this information to build a short {language} brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

# put the two function above together
def create_brochure(company_name, url, language="English", tone ="humorous, entertaining, jokey"):
    system_prompt = f"""You are an assistant that analyzes the contents of several relevant pages from a company website \
    and creates a {tone} about the company for prospective customers, investors and recruits. Respond in markdown.\
    Include details of company culture, customers, product pricing and careers/jobs if you have the information.

    """

    stream= openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, language=language)}
          ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="English", tone="humorous, entertaining, jokey")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'pricing page', 'url': 'https://www.brooklynkids.org/tickets/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'contact page', 'url': 'https://www.brooklynkids.org/contact/'}]}


# Welcome to the Brooklyn Children's Museum!

### 🌟 Who Are We? 🌟
Located in the wonderful (and slightly quirky) Brooklyn, we are the Brooklyn Children’s Museum, the OG of children’s museums! Established way back in 1899, we pride ourselves on being a place to *play*, *learn*, and *discover*! Our mission is to inspire curiosity and creativity in children and families—because why should only adults have all the fun?

### 🎠 Our Attractions 🎠
- **Exhibits**: Get ready for mind-blowing adventure in our interactive exhibits, where every day feels like a holiday! Explore the *Totally Tots* area, where no one is too small to have big fun! 
- **ColorLab Art Studio**: Unleash your inner Picasso and leave the mess to us!  
- **STEM Makerspace**: Science? Technology? Engineering? Math? All in one place! Think of it as Hogwarts, but for future scientists and inventors. 
- **Movies & Cartoons**: Sit back, relax, and enjoy films that will make you laugh, cry, and everything in-between (in the best possible way).

### 🎉 **Birthday Bashes & Rentals Galore!** 🎉
Celebrating another trip around the sun? BCM is the perfect place for birthday parties! We promise fun so big you’ll need a map to navigate it! Plus, if you happen to be thinking about renting space for an event, we’ve got flexibility that would put yoga instructors to shame!

### 💰 **Membership & Pricing** 💰
Why pay for one visit when you can spend the whole year hanging out with us? Join the BCM family with our membership deals! And for a limited time, memberships are 14% off—so hurry up and snag one before we start celebrating with confetti!

### 🌍 **Where to Find Us** 🌍
- **Address**: 145 Brooklyn Avenue, Brooklyn, NY 11213, USA  
- **Hours**: Wednesday-Sunday: 10 AM – 5 PM (Monday and Tuesday we rest—because even museums need a spa day!)

### 🎓 **Careers & Community** 🎓
Want to be a part of a place where magic happens every day? We’re always looking for vibrant individuals to join our team! If you have a love for children, a knack for creativity, and a talent for dodging flying LEGO pieces, apply today!

---

### 👀 **In Conclusion:**
So, whether you're looking for a fun family day out, an exciting place to host an event, or a position that keeps you young at heart, the Brooklyn Children's Museum welcomes you with open arms. Come visit us and experience the wonder—after all, we believe that the more you play, the more you learn (and the more fun you have)!

See you soon! 🎈

In [10]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="English", tone="formal")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'company page', 'url': 'https://www.brooklynkids.org/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}]}


# Brooklyn Children's Museum

**Welcome to Brooklyn Children's Museum**  
Where learning and fun go hand in hand! 

---

## **About Us**

Founded in 1899, the Brooklyn Children's Museum is the first children's museum in the world. Our mission is to inspire the curiosity and creativity of children and families through dynamic exhibitions, educational programs, and community outreach initiatives. We believe in the transformative power of play and discovery that fosters lifelong learning.

### **Culture**

At BCM, we foster an inclusive and engaging environment that values creativity, diversity, and community engagement. Our staff is dedicated to creating memorable experiences that celebrate learning and exploration. We prioritize a child-centered approach, ensuring that both children and their families feel welcome and excited to discover the world around them.

## **Our Offerings**

### **Exhibits**
Explore a variety of interactive exhibits such as:
- **Totally Tots**: A safe space for our youngest explorers.
- **Neighborhood Nature**: Discover the beauty of the natural world.
- **AirMaze**: An innovative play area that challenges and educates.

### **Education Programs**
We offer a variety of educational programs, including:
- **School Visits**
- **Summer Camp Program**
- **After School Programs**
- **BCM @ Home Activity Kits**

### **Events & Activities**
Join us for numerous events like:
- **Black Future Festival**
- **Community Access Thursdays**

## **Pricing and Memberships**

- **Ticket Prices**: Visit our website for detailed pricing.
- **Membership**: Become a member to enjoy unlimited visits and exclusive events. Take advantage of our current promotion of 14% off all memberships (available for a limited time)!

## **Careers at BCM**

We are always looking for passionate individuals to join our team. Explore a variety of roles in education, operations, and visitor services. BCM is committed to professional development and offers resources for employee growth.

---

## **Visit Us**

**Location**:  
Brooklyn Children’s Museum  
145 Brooklyn Avenue, Brooklyn, NY 11213, USA  

**Hours**:  
Wednesday to Sunday: 10 am – 5 pm  
Closed Monday and Tuesday  

---

## **Get in Touch**

For more information, visit our website or contact us through email or phone. Don't forget to **join** our mailing list for updates on upcoming events and special offers!

**[Website](https://www.brooklynkids.org)**  
**Phone**: (Insert Phone Number)  
**Email**: (Insert Email Address)  

**Join us at BCM, where curiosity blossoms into lifelong memories!**  


In [11]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="Chinese", tone="formal")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}, {'type': 'contact page', 'url': 'https://www.brooklynkids.org/contact/'}]}



# 布鲁克林儿童博物馆简介

欢迎来到布鲁克林儿童博物馆，一个充满乐趣和学习机会的地方！我们致力于为儿童和家庭提供丰富多彩的展览、教育项目和互动体验。

## 我们的使命
布鲁克林儿童博物馆的使命是通过探索和发现，激发儿童对文化、艺术和科学的好奇心。我们相信通过实践学习和玩耍，儿童能够更好地理解世界。

## 主要展览与项目
- **ColorLab艺术工作室**: 让孩子们进行创造性的艺术探索，激发他们的想象力。
- **STEM创客空间**: 专为年轻科学家和工程师设计，提升他们在科学、技术、工程和数学方面的技巧。
- **Totally Tots**: 特别为幼儿打造的互动小天地。
- **邻里自然展**: 与社区的自然环境亲密接触，拓宽孩子们对自然的理解。

## 活动与租赁服务
我们提供丰富的活动策划，包括适合所有年龄段的生日派对和空间租赁服务。无论是个人庆祝还是学校团体活动，我们都能提供完美的场地！

## 实惠的票价和会员计划
- **单日门票**: 有竞争力的定价，确保所有家庭都能在经济上负担得起。
- **会员计划**: 加入我们，享受年度无限制免费入馆、独家活动和特别优惠。

## 工作机会
我们欢迎热爱教育与儿童发展的专业人士加入我们的团队。可浏览我们的[招聘页面](链接)了解更多信息。

## 公司文化
在布鲁克林儿童博物馆，我们倡导包容多样的文化环境。我们重视每一位员工的声音，并鼓励团队合作，共同创造积极和富有创意的工作氛围。

## 联系我们
地址: 布鲁克林大道145号，布鲁克林，纽约 11213  
电话: 请访问我们的[官方网站](链接)获取更多信息。  
来访时间: 每周三至周日 早上10点至下午5点（周一及周二闭馆）。

感谢您支持布鲁克林儿童博物馆，期待您的光临与参与！

---



In [12]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="Thai", tone="formal")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}]}


# Brooklyn Children's Museum: A World of Learning and Fun for Every Child

## About Us
The **Brooklyn Children's Museum** (BCM) is a vibrant, interactive space designed to spark curiosity and creativity in children of all ages. Established with a mission to enrich the lives of children through learning and play, our museum offers a range of engaging exhibits and educational programs.

## Our Values and Culture
At BCM, we believe in fostering a fun and inclusive environment where children can explore, create, and learn. Our commitment to diversity and accessibility is reflected in our programs, ensuring every child feels welcome and valued. We strive to be a community hub that empowers families and encourages lifelong learning.

## What We Offer
### Exhibits
- **ArtRink**: A space where artistic expression comes to life.
- **Neighborhood Nature**: An interactive exploration of local ecology.
- **Totally Tots**: Designed especially for young explorers.

### Programs
- **ColorLab Art Studio**: Hands-on art activities for all ages.
- **STEM Makerspace**: A creative space promoting science and technology through interactive experiences.
- **Community Access Thursdays**: Offering special access to exhibits and programs for everyone.

### Educational Opportunities
Our education initiatives include:
- **School Visits**: Tailored experiences for classes, rich in learning and fun.
- **Summer Camp Program**: Engaging summer activities for young learners.
- **BCM @ Home**: Activity kits designed for continued learning outside the museum.

## Special Events
Join us for various events throughout the year, including the **Black Future Festival** and themed movie screenings! Check our calendar for upcoming activities.

## Pricing and Membership
Become a member to enjoy unlimited visits and family events throughout the year. We currently offer a limited-time **14% off all memberships** – a fantastic opportunity for families to get involved and make the most of what BCM has to offer!

## Work With Us
If you’re passionate about education and children’s development, consider a career at BCM. We offer various employment and internship opportunities where you can make a meaningful impact.

## Visit Us
We’re open  
**Wednesday to Sunday, 10 am - 5 pm**  
**Location**: 145 Brooklyn Avenue, Brooklyn, NY 11213, USA

### Contact Us
For more information about our activities, programs, or membership, visit our website or reach out to our friendly staff. We look forward to welcoming you to the Brooklyn Children's Museum!

---

### Join the BCM Community
Experience the joy of discovery and the excitement of learning at the Brooklyn Children's Museum. Together, let’s inspire the next generation!

In [13]:
create_brochure("Brooklyn children's museum", "https://www.brooklynkids.org/", language="Thai", tone="playful, humorous, entertaining")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'contact page', 'url': 'https://www.brooklynkids.org/contact/'}]}



# Discover the Brooklyn Children's Museum! 🎈

### The Ultimate Playground of Learning and Fun!

Welcome to the Brooklyn Children's Museum (BCM), where imagination takes flight, and every visit is a new adventure! Located at 145 Brooklyn Avenue, the museum opens its doors (and its heart) to kids and families from Wednesday to Sunday, 10 AM to 5 PM. So grab your little Einsteins and prepare for a day packed with exploration!

---

## 🧩 **What’s Inside?**

**Exhibit Extravaganza!**  
Dive into our **ColorLab Art Studio**, where creativity knows no bounds! Unleash your inner Picasso while experimenting with colors and textures like a true artist.  
For the little builders, our **STEM Makerspace** is a wonderland filled with puzzles, robotics, and all things science-y! Just a warning: Most visitors leave in a flurry of glitter and genius!

**Don't miss our crowd favorites:**
- **Totally Tots**: A toddler paradise!
- **Neighborhood Nature**: Outdoor fun without the bugs!

---

## 🌟 **Celebrate at BCM!**

Feeling like throwing a party with the finest festivities? Our birthday parties are the talk of the town! Whether baby or big kids, we have packages as colorful as a birthday cake. Your guests will leave with hearts full of joy and hands full of goody bags. 🎂

---

## 🎓 **Learning - It's Not Just for Grown-Ups!**

In addition to fun, we take education seriously! Kids can immerse themselves in hands-on learning through our:
- **School Visits**: Transform a day of regular classes into an exciting adventure!
- **Summer Camp Program**: A perfect mix of learning and play for those sun-soaked days! ☀️



---

## 💰 **Membership Perks!**

Want a year-long pass to fun? Become a BCM member and enjoy UNLIMITED free admission, special events, and exclusive discounts! *Pssst… right now, you can grab a 14% OFF membership!*

---

## 👨‍👩‍👧 **Our Family Culture**

At BCM, we’re like a big, happy family! 🎉 Our staff is dedicated to creating a warm, inviting atmosphere where both kids and parents can feel excited to explore, learn, and grow together. We believe in the power of play as a serious business, and we want YOU to be part of our playful community!

---

## 📢 **Join Us!**

Have experience in education or want to learn? We're often on the lookout for enthusiastic new team members! Check our employment page for opportunities to become a BCM superstar!

**Contact Us:** For more information or to book your spot for a visit, contact us at [contact@brooklynchildrensmuseum.org](mailto:contact@brooklynchildrensmuseum.org). 

---

So, what are you waiting for? Come play, create, and learn at the Brooklyn Children's Museum—where every day is a new adventure! **We can't wait to meet YOU!**

---
*Follow us on this adventure by subscribing to our newsletter for updates on events, discounts, and more!*


# updates: Add Web UI 

In [22]:
def create_brochure(company_name, url, language="English", tone ="humorous, entertaining, jokey"):
    system_prompt = f"""You are an assistant that analyzes the contents of several relevant pages from a company website \
    and creates a {tone} about the company for prospective customers, investors and recruits. Respond in markdown.\
    Include details of company culture, customers, product pricing and careers/jobs if you have the information.

    """

    stream= openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, language=language)}
          ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
        
view = gr.Interface(
    fn=create_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Textbox(label="Language"),
        gr.Textbox(label="Tone")],
        outputs=[gr.Markdown(label="Brochure:")]
,
flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'about page', 'url': 'https://www.brooklynkids.org/about/'}, {'type': 'careers page', 'url': 'https://www.brooklynkids.org/employment/'}, {'type': 'visit page', 'url': 'https://www.brooklynkids.org/visit/'}, {'type': 'programs page', 'url': 'https://www.brooklynkids.org/programs/'}, {'type': 'contact page', 'url': 'https://www.brooklynkids.org/contact/'}, {'type': 'events page', 'url': 'https://www.brooklynkids.org/events/'}]}



# 🌟 Welcome to the Brooklyn Children's Museum! 🎉

## **About Us**  
At the Brooklyn Children’s Museum (BCM), we believe in the power of play and learning! Nestled in the heart of Brooklyn, our museum is a vibrant and welcoming space where children and families can explore, create, and discover together. 

### **Our Mission**  
Inspired by the energy and diversity of Brooklyn, BCM strives to provide meaningful experiences that ignite curiosity, creativity, and social engagement. We are committed to being a safe, inclusive space for children to learn through play.

## **Fun for Everyone! 🎈**  
BCM offers a diverse array of exhibits and programs that cater to every little explorer! Some of our highlights include:

- **ColorLab Art Studio**: Unleash your inner artist and create colorful masterpieces!
- **STEM Makerspace**: Dive into hands-on science, technology, engineering, and math activities.
- **Totally Tots**: A toddler haven full of interactive fun!

Our exciting **Community Access Thursdays** make it even easier for everyone to join the fun! 

## **Birthday Party Bliss 🎂**  
Celebrate your child’s special day at BCM! Our birthday parties offer a unique and memorable way to celebrate with friends, surrounded by creativity and fun. 

## **Learning Through Play 🧠**  
Whether you're visiting with a school group, a summer camp, or just looking for an educational outing, BCM’s engaging programs promote learning through play. Our **After School Program** and **BCM @ Home Activity Kits** ensure that the fun and learning continue even beyond your visit!

## **Pricing & Memberships 💸**  
Joining the BCM family is easy and rewarding! Become a member to enjoy **unlimited free admission** to exhibits and events, exclusive member programs, and discounts on birthday parties and rentals. 

- **General Admission**: Check our website for pricing details.
- **Membership**: Sign up for access to a full year of fun!

## **Join Our Community! 🤝**  
Are you passionate about educational engagement and community development? Explore exciting **career opportunities** at BCM and be part of a team dedicated to inspiring future generations!

### **Get to Know Us!**  
- **Location**: 145 Brooklyn Avenue, Brooklyn, NY 11213
- **Hours**: Wednesday to Sunday - 10 am to 5 pm (Closed Monday and Tuesday)

## **Stay Connected! 📬**  
Never miss out on the fun! Subscribe to our newsletter to stay updated on programs, events, and more!

Join us in creating unforgettable memories and a love for learning at the Brooklyn Children’s Museum. We can’t wait to see you!

